In [64]:
import ?

In [ ]:
python run.py --spotter_config ./spotter-v2/PALEJUN/configs/PALEJUN/SynthMap/SynthMap_Polygon.yaml

'mapkurator-spotter/spotter-v2/configs/PALEJUN/Finetune/Base-SynthMap-Polygon.yaml'

--module_text_spotting turns on the spotting module in this run
--sample_map_csv_path stores the metadata of the input map, a sample file can be found here.
--text_spotting_model_dir switches to the model directory



In [ ]:
gpu_id = 0
spotter_model = 'spotter-v2'
spotter_config = args.spotter_config

In [ ]:
# ------------------------- Text Spotting (patch level) ------------------------------
if module_text_spotting:
    assert os.path.exists(spotter_config), "Config file for spotter must exist!"
    os.chdir(text_spotting_model_dir) 
    os.system("python setup.py build develop 1> /dev/null")

    for index, record in sample_map_df.iterrows():

        external_id = record.external_id
        if "sample_map_path" in locals():
            if external_id not in external_id_to_img_path_dict:
                error_reason_dict[external_id] = {'img_path':None, 'error':'key not in external_id_to_img_path_dict'} 
                continue 
            img_path = external_id_to_img_path_dict[external_id]
        else:
            img_path = sample_map_df['external_id'].iloc[index]

        map_name = os.path.basename(img_path).split('.')[0]
        # print(map_name)

        map_spotting_output_dir = os.path.join(spotting_output_dir, map_name)
        
        if not os.path.isdir(map_spotting_output_dir):
            os.makedirs(map_spotting_output_dir)
        else:
            num_existing_json = len(glob.glob(os.path.join(map_spotting_output_dir, '*.json')))
            num_existing_images = len(glob.glob(os.path.join(cropping_output_dir, map_name, '*jpg')))
            if num_existing_json == num_existing_images:
                continue
            else:
                print(f'{index}/{len(sample_map_df)}: Re-run spotting for map {map_name}')
                import shutil
                shutil.rmtree(map_spotting_output_dir)
                os.makedirs(map_spotting_output_dir)        

        if spotter_model in ['testr', 'spotter-v2', 'palette']:
            run_spotting_command = f'CUDA_VISIBLE_DEVICES={gpu_id} python tools/inference.py --config-file {spotter_config} --output_json --input {os.path.join(cropping_output_dir,map_name)} --output {map_spotting_output_dir}'
        else:
            raise NotImplementedError
        
        # print(run_spotting_command)
        run_spotting_command  += ' 1> /dev/null'
    
        exe_ret = execute_command(run_spotting_command, if_print_command)  
        if "sample_map_path" in locals():   
            if 'error' in exe_ret:
                error = exe_ret['error']
                error_reason_dict[external_id] = {'img_path':img_path, 'error': error } 
        
        # elif 'time_usage' in exe_ret:
        #     time_usage = exe_ret['time_usage']
        #     time_usage_dict[external_id]['spotting'] = time_usage
        # else:
        #     raise NotImplementedError

        logging.info(f'{index}/{len(sample_map_df)}: Done text spotting for {map_name}')
        
# time_text_spotting = time.time()
    